# Assignment 3

In [1]:
import pandas as pd
import numpy as np

# Load MovieLens 100K dataset into a dataframe of pandas
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', sep='\t', names=names)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [2]:
# Select 500 most active users and 500 most active items from the dataset
n_most_active_users = 500
n_most_active_items = 500

user_ids = df.groupby('user_id').count().sort_values(by='rating', ascending=False).head(n_most_active_users).index
item_ids = df.groupby('item_id').count().sort_values(by='rating', ascending=False).head(n_most_active_items).index
df = df[(df['user_id'].isin(user_ids)) & (df['item_id'].isin(item_ids))]

In [3]:
# Map new internal ID for items
i_ids = df['item_id'].unique().tolist()
item_dict = dict(zip(i_ids, [i for i in range(len(i_ids))]))
df['item_id'] = df['item_id'].map(item_dict)

# Split Dataset

In [4]:
# The number of training users and active users
n_training_users = 300
n_active_users = n_most_active_users - n_training_users

# The number of GIVEN ratings for active users
GIVEN = 20

# Randomly select users from the most active users as training set
random_uids = np.random.choice(df.user_id.unique(), n_training_users, replace=False)
train_df = df[df['user_id'].isin(random_uids)]
# Map new internal ID for all users in the training set
u_ids = train_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
train_df['user_id'] = train_df['user_id'].map(user_dict)

# The rest of users are active users for testing
remain_df = df[~df['user_id'].isin(random_uids)]
# Map new internal ID for all active users
u_ids = remain_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
remain_df['user_id'] = remain_df['user_id'].map(user_dict)

# Randomly select GIVEN ratings for active users
active_df = remain_df.groupby('user_id').sample(n=GIVEN, random_state=1024)

test_df = remain_df[~remain_df.index.isin(active_df.index)]

<ipython-input-4-fb140fbf3b12>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['user_id'] = train_df['user_id'].map(user_dict)
<ipython-input-4-fb140fbf3b12>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remain_df['user_id'] = remain_df['user_id'].map(user_dict)


In [5]:
# Convert the format of datasets to matrices
df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_training_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_training_users), 'rating': 0})
train_ds = df_zeros.merge(train_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_active_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_active_users), 'rating': 0})
active_ds = df_zeros.merge(active_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')
test_ds = df_zeros.merge(test_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

train_ds, active_ds, test_ds

(item_id  0    1    2    3    4    5    6    7    8    9    ...  490  491  492  \
 user_id                                                    ...                  
 0        3.0  0.0  0.0  0.0  0.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0  0.0   
 1        0.0  2.0  0.0  4.0  0.0  4.0  4.0  0.0  0.0  2.0  ...  0.0  4.0  4.0   
 2        0.0  0.0  4.0  4.0  4.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0  0.0   
 3        4.0  0.0  0.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 4        0.0  0.0  0.0  0.0  5.0  0.0  0.0  0.0  0.0  4.0  ...  0.0  0.0  0.0   
 ...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 295      4.0  0.0  4.0  0.0  0.0  4.0  3.0  4.0  0.0  0.0  ...  0.0  0.0  0.0   
 296      0.0  0.0  0.0  0.0  0.0  0.0  0.0  5.0  4.0  5.0  ...  0.0  0.0  4.0   
 297      0.0  0.0  5.0  4.0  0.0  1.0  0.0  0.0  0.0  1.0  ...  0.0  0.0  0.0   
 298      0.0  0.0  0.0  3.0  0.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0  5.0   
 299      0.0  0

In [6]:
# Predicting All Missing Data in training set
imputed_train_ds = train_ds.values.copy()

# Your implementation to predict the missing values
(Put all your implementation for your algorithm in the following cell only to handle the missing values; )

In [7]:
## Put all your implementation for your solutioin in this cell only to predict the missing values; 
## NOTE 1: DO NOT change anything in the rest of the cells in this framework, 
## otherwise the changes might cause errors and make your implementation invalid.

## Note 2: 
## The user-item rating matrix is imputed_train_ds, 
## and the missing values are those 0s in imputed_train_ds. 
## You are required to predict them by using the solution in the given report. 

## The following parameters are required in the given report, 
## which is named "Effective Missing Data Prediction for Collaborative Filtering", 
## and you will need to use them. But, please do not change their values. 
LAMBDA = 0.7    # λ
GAMMA = 10      # γ
DELTA = 10      # δ
ITA = 0.7       # η
THETA = 0.7     # θ
EPSILON = 1e-9

# my code start here 

n_users = train_ds.shape[0]
n_items = train_ds.shape[1]

# Compute Pearson Correlation Coefficient of All Pairs of Users in training set
np_user_pearson_corr = np.zeros((n_users, n_users))
similar_user_list = []

# Compute Pearson Correlation Coefficient for Each Pair of Users in Training Dataset
for i, user_i_vec in enumerate(train_ds.values):
    for j, user_j_vec in enumerate(train_ds.values):

        # ratings corated by the current pair of users
        mask_i = user_i_vec > 0
        mask_j = user_j_vec > 0

        # corrated item index, skip if there are no corrated ratings
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        if len(corrated_index) == 0:
            continue

        # average value of user_i_vec and user_j_vec
        mean_user_i = np.sum(user_i_vec) / (np.sum(np.clip(user_i_vec, 0, 1)) + EPSILON)
        mean_user_j = np.sum(user_j_vec) / (np.sum(np.clip(user_j_vec, 0, 1)) + EPSILON)

        # compute pearson corr
        user_i_sub_mean = user_i_vec[corrated_index] - mean_user_i
        user_j_sub_mean = user_j_vec[corrated_index] - mean_user_j

        r_ui_sub_r_i_sq = np.square(user_i_sub_mean)
        r_uj_sub_r_j_sq = np.square(user_j_sub_mean)

        r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_r_i_sq))
        r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_r_j_sq))

        sim = np.sum(user_i_sub_mean * user_j_sub_mean) / (r_ui_sum_sqrt * r_uj_sum_sqrt + EPSILON)

        # significance weighting
        weighted_sim = (min(len(corrated_index), GAMMA) / GAMMA) * sim
        
        np_user_pearson_corr[i][j] = weighted_sim
        
        if weighted_sim > THETA:
            if i != j:
                similar_user_list.append(i)
            else:
                pass
        else:
            pass

            
            
# Compute Pearson Correlation Coefficient of All Pairs of Items in training set          
np_item_pearson_corr = np.zeros((n_items, n_items))
similar_item_list = []

# Compute Pearson Correlation Coefficient for Each Pair of Items in Training Dataset
for i, item_i_vec in enumerate(train_ds.T.values):
    for j, item_j_vec in enumerate(train_ds.T.values):

        # ratings corated by the current pair of items
        mask_i = item_i_vec > 0
        mask_j = item_j_vec > 0

        # corrated index, skip if there are no corrated ratings
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        if len(corrated_index) == 0:
            continue

        # average value of item_i_vec and item_j_vec
        mean_item_i = np.sum(item_i_vec) / (np.sum(np.clip(item_i_vec, 0, 1)) + EPSILON)
        mean_item_j = np.sum(item_j_vec) / (np.sum(np.clip(item_j_vec, 0, 1)) + EPSILON)

        # compute pearson corr
        item_i_sub_mean = item_i_vec[corrated_index] - mean_item_i
        item_j_sub_mean = item_j_vec[corrated_index] - mean_item_j

        r_ui_sub_ri_sq = np.square(item_i_sub_mean)
        r_uj_sub_rj_sq = np.square(item_j_sub_mean)

        r_ui_sub_ri_sq_sum_sqrt = np.sqrt(np.sum(r_ui_sub_ri_sq))
        r_uj_sub_rj_sq_sum_sqrt = np.sqrt(np.sum(r_uj_sub_rj_sq))

        sim = np.sum(item_i_sub_mean * item_j_sub_mean) / (r_ui_sub_ri_sq_sum_sqrt * r_uj_sub_rj_sq_sum_sqrt + EPSILON)

        # significance weighting
        weighted_sim = (min(len(corrated_index), DELTA) / DELTA) * sim
        
        np_item_pearson_corr[i][j] = weighted_sim
        
        if weighted_sim > ITA: 
            if i != j:
                similar_item_list.append(i)
            else:
                pass
        else:
            pass




            
# Predicting Missing Values in imputed_train_ds 


# Creating user , item and user-item prediction matrix 
np_user_prediction = np.zeros((n_users, n_items))
np_item_prediction = np.zeros((n_users, n_items))
np_user_item_prediction = np.zeros((n_users, n_items))

#  Compute Pearson Correlation Coefficient of All Pairs of Users in imputed training set
for (i,j),ratings in np.ndenumerate(imputed_train_ds):
    
    # for condition S(u) ̸= ∅ ∧ S(i) = ∅ ,which means user have similar user and item does not have similar item
    if i in np.unique(similar_user_list) and j not in np.unique(similar_item_list):
        # Ignoring item whose rating is already provided by user
        if ratings == 0:

            
            #==================user-based==================#
            # the coefficient values of similar users
            sim_val = np_user_pearson_corr[i][similar_user_list]
            
            # the average value of the current user's ratings
            sim_users = train_ds.values[similar_user_list]
            
            user_mean = np.sum(train_ds.values[i]) / (np.sum(np.clip(train_ds.values[i], 0, 1)) + EPSILON)

            sim_user_mean = np.sum(sim_users, axis=1) / (np.sum(np.clip(sim_users, 0, 1), axis=1) + EPSILON)

            # select the users who rated item j
            mask_rated_j = sim_users[:, j] > 0
            
            sim_r_sum_mean = sim_val[mask_rated_j] * (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])
            
            np_user_prediction[i][j] = user_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val[mask_rated_j]) + EPSILON)
            np_user_prediction[i][j] = np.clip(np_user_prediction[i][j], 0, 5)
            
            # predicting missing data 
            imputed_train_ds[i][j] = np_user_prediction[i][j]
        else:
            pass

        
        
    # for condition S(u) = ∅ ∧ S(i) ̸= ∅,,which means user does not  have similar user and item  have similar item       
    elif i not in np.unique(similar_user_list) and j in np.unique(similar_item_list):
        # Ignoring item whose rating is already provided by user
        if ratings == 0:

            
            #==================item-based==================#
            # the coefficient values of similar items
            sim_val = np_item_pearson_corr[j][similar_item_list]

            # the average value of the current item's ratings
            sim_items = train_ds.T.values[similar_item_list]
            item_mean = np.sum(train_ds.T.values[j]) / (np.sum(np.clip(train_ds.T.values[j], 0, 1)) + EPSILON)
            sim_item_mean = np.sum(sim_items, axis=1) / (np.sum(np.clip(sim_items, 0, 1), axis=1) + EPSILON)

            sim_r_sum_mean = sim_val * (sim_items[:, i] - sim_item_mean) 

            # filter unrated items
            w = np.clip(sim_items[:, i], 0, 1)
            sim_r_sum_mean *= w

            np_item_prediction[i][j] = item_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val * w) + EPSILON)    
            np_item_prediction[i][j] = np.clip(np_item_prediction[i][j], 0, 5)
            
            # predicting missing data 
            imputed_train_ds[i][j] = np_item_prediction[i][j]
        else:
            pass

        
            
            
    # for condition S(u) ̸= ∅ ∧ S(i) ̸= ∅,which means user  have similar user and item  have similar item        
    elif i in np.unique(similar_user_list) and j in np.unique(similar_item_list):
        # Ignoring item whose rating is already provided by user
        if ratings == 0:

            
            #==================user-based==================#
            # the coefficient values of similar items
            sim_user_val = np_user_pearson_corr[i][similar_user_list]
            
            # the average value of the current user's ratings
            sim_users = train_ds.values[similar_user_list]
            user_mean = np.sum(train_ds.values[i]) / (np.sum(np.clip(train_ds.values[i], 0, 1)) + EPSILON)
            sim_user_mean = np.sum(sim_users, axis=1) / (np.sum(np.clip(sim_users, 0, 1), axis=1) + EPSILON)
            
            # select the users who rated item j
            mask_rated_j = sim_users[:, j] > 0

            sim_user_sum_mean = sim_user_val[mask_rated_j] * (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])
            np_user_prediction[i][j] = user_mean + np.sum(sim_user_sum_mean) / (np.sum(sim_user_val[mask_rated_j]) + EPSILON)
            np_user_prediction[i][j] = np.clip(np_user_prediction[i][j], 0, 5)
            

            
            #==================item-based==================#
            # the coefficient values of similar items
            sim_items_val = np_item_pearson_corr[j][similar_item_list]

            # the average value of the current item's ratings
            sim_items = train_ds.T.values[similar_item_list]
            item_mean = np.sum(train_ds.T.values[j]) / (np.sum(np.clip(train_ds.T.values[j], 0, 1)) + EPSILON)
            sim_item_mean = np.sum(sim_items, axis=1) / (np.sum(np.clip(sim_items, 0, 1), axis=1) + EPSILON)

            # sim(u, v) * (r_v - mean_v)
            sim_items_sum_mean = sim_items_val * (sim_items[:, i] - sim_item_mean) 

            # filter unrated items
            w = np.clip(sim_items[:, i], 0, 1)
            sim_items_sum_mean *= w

            np_item_prediction[i][j] = item_mean + np.sum(sim_items_sum_mean) / (np.sum(sim_items_val * w) + EPSILON)    
            np_item_prediction[i][j] = np.clip(np_item_prediction[i][j], 0, 5)
            
            

             #==================user-item-based==================#
            np_user_item_prediction[i][j] = LAMBDA * np_user_prediction[i][j] + (1-LAMBDA) * np_item_prediction[i][j]
            
            # predicting missing data 
            imputed_train_ds[i][j] = np_user_item_prediction[i][j]
            
        else:
            pass

        
        
        
    # for condition S(u) = ∅ ∧ S(i) = ∅,which means user does not have similar user and item does not have similar item   
    elif  i not in np.unique(similar_user_list) and j not in np.unique(similar_item_list):
        imputed_train_ds[i][j] = 0
        
    else:
        pass
           

# Evaluation

### Compute Pearson Correlation Coefficient of All Pairs of Items between active set and imputed training set

In [8]:
imputed_train_ds = pd.DataFrame(imputed_train_ds)
imputed_train_ds

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,3.000000,3.969267,0.000000,4.289162,4.074694,4.679482,4.239571,4.000000,3.209258,0.000000,...,3.387377,0.000000,3.969453,3.922343,2.833656,2.426839,4.000000,3.745867,3.000000,3.594393
1,4.502529,2.000000,4.459209,4.000000,3.814617,4.000000,4.000000,3.896045,3.400430,2.000000,...,2.389422,4.000000,4.000000,4.260441,3.000000,3.000000,1.315622,3.835841,2.610957,3.009822
2,4.474650,4.048239,0.000000,4.000000,4.000000,3.385149,3.934616,4.000000,3.419671,0.000000,...,3.334185,0.000000,3.855872,3.744351,3.434274,2.940300,2.670019,3.657026,5.000000,3.478824
3,4.000000,3.620069,4.865996,2.000000,3.599204,2.791688,4.368417,3.872830,3.999935,3.842566,...,2.624301,2.723569,3.907082,4.164627,3.510754,3.497247,2.314229,4.112331,1.874011,3.594332
4,4.427639,4.106334,4.524672,4.475162,5.000000,3.846605,4.225011,4.457174,3.780521,4.000000,...,2.756791,1.635554,4.218843,4.595951,2.836338,2.112661,2.095117,4.122644,2.939534,3.674478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,4.000000,3.559687,0.000000,3.796356,3.381483,4.000000,3.000000,4.000000,3.786467,0.000000,...,2.750471,0.000000,4.010568,3.621197,2.986281,2.841546,5.000000,3.631558,3.355794,3.304517
296,4.328517,3.790302,0.000000,4.179794,4.308024,3.669592,5.000000,5.000000,4.000000,0.000000,...,3.277177,0.000000,4.000000,3.983613,4.000000,2.000000,3.000000,3.982252,3.339346,3.291259
297,3.483959,2.670491,5.000000,4.000000,2.878698,1.000000,3.242903,2.988495,2.419147,1.000000,...,1.716804,1.536514,2.943046,3.557816,2.141971,0.805823,0.685688,3.205764,1.000000,2.563450
298,3.546173,3.096556,0.000000,3.000000,2.465742,2.258197,4.534548,4.000000,2.435301,0.000000,...,2.563443,0.000000,5.000000,3.502753,3.139623,3.000000,0.129561,3.067522,5.000000,2.328342


In [9]:
active_user_pearson_corr = np.zeros((active_ds.shape[0], train_ds.shape[0]))

# Compute Pearson Correlation Coefficient of All Pairs of Users between active set and imputed training set
for i, user_i_vec in enumerate(active_ds.values):
    for j, user_j_vec in enumerate(imputed_train_ds.values):
        
        # ratings corated by the current pair od users
        mask_i = user_i_vec > 0
        mask_j = user_j_vec > 0

        # corrated item index, skip if there are no corrated ratings
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        if len(corrated_index) == 0:
            continue

        # average value of user_i_vec and user_j_vec
        mean_user_i = np.sum(user_i_vec) / (np.sum(np.clip(user_i_vec, 0, 1)) + EPSILON)
        mean_user_j = np.sum(user_j_vec) / (np.sum(np.clip(user_j_vec, 0, 1)) + EPSILON)

        # compute pearson corr
        user_i_sub_mean = user_i_vec[corrated_index] - mean_user_i
        user_j_sub_mean = user_j_vec[corrated_index] - mean_user_j

        r_ui_sub_r_i_sq = np.square(user_i_sub_mean)
        r_uj_sub_r_j_sq = np.square(user_j_sub_mean)

        r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_r_i_sq))
        r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_r_j_sq))

        sim = np.sum(user_i_sub_mean * user_j_sub_mean) / (r_ui_sum_sqrt * r_uj_sum_sqrt + EPSILON)

        # significance weighting
        weighted_sim = (min(len(corrated_index), GAMMA) / GAMMA) * sim

        active_user_pearson_corr[i][j] = weighted_sim

active_user_pearson_corr

array([[ 4.56613538e-01,  1.59676125e-01,  3.39051432e-01, ...,
         2.55766845e-01, -1.50321042e-01,  4.00843692e-01],
       [ 2.61674426e-01,  2.60916802e-01, -1.72370176e-01, ...,
        -7.60328532e-02, -1.25902948e-01,  4.99035204e-01],
       [-8.84366639e-02, -7.86347495e-02,  2.03618939e-01, ...,
        -1.38066815e-01, -5.78359588e-04, -1.05828306e-01],
       ...,
       [ 2.32852934e-01,  4.06847356e-01, -8.29509982e-02, ...,
         2.61146762e-01,  1.45599384e-01,  2.00111694e-01],
       [ 5.94828739e-01, -3.85784022e-02,  1.39116163e-01, ...,
         1.42348263e-01, -2.25861606e-01, -1.55675673e-01],
       [ 3.93320408e-01,  1.56492879e-01,  5.16038865e-01, ...,
         6.25967140e-01,  1.84677964e-01,  6.74612050e-01]])

## Predict Ratings of Testing Set

In [10]:
K = 10

test_ds_pred = np.zeros_like(test_ds.values)

for (i, j), rating in np.ndenumerate(test_ds.values):

    if rating > 0:

        sim_user_ids = np.argsort(active_user_pearson_corr[i])[-1:-(K + 1):-1]

        #==================user-based==================#
        # the coefficient values of similar users
        sim_val = active_user_pearson_corr[i][sim_user_ids]

        # the average value of the current user's ratings
        sim_users = imputed_train_ds.values[sim_user_ids]
        user_mean = np.sum(active_ds.values[i]) / (np.sum(np.clip(active_ds.values[i], 0, 1)) + EPSILON)
        sim_user_mean = np.sum(sim_users, axis=1) / (np.sum(np.clip(sim_users, 0, 1), axis=1) + EPSILON)

        # select the users who rated item j
        mask_rated_j = sim_users[:, j] > 0
        
        # sim(u, v) * (r_vj - mean_v)
        sim_r_sum_mean = sim_val[mask_rated_j] * (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])
        
        user_based_pred = user_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val[mask_rated_j]) + EPSILON)
        user_based_pred = np.clip(user_based_pred, 0, 5)

        test_ds_pred[i][j] = user_based_pred
        
test_ds_pred


array([[0.        , 0.        , 0.        , ..., 0.        , 3.32353306,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 4.01574701,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [4.1899937 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 3.23144726, 0.        , ..., 0.        , 0.        ,
        0.        ]])

## Compute MAE and RMSE

In [11]:
# MAE
MAE = np.sum(np.abs(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1))

# RMSE
RMSE = np.sqrt(np.sum(np.square(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1)))

print("MAE: {}, RMSE: {}" .format(MAE, RMSE))

MAE: 0.7597375140784667, RMSE: 0.9800963926477396
